<a href="https://colab.research.google.com/github/MathieuFan22/Mirada-Examen-ML-M2-ISPM/blob/Tsiky/PoketraFinday-Mirada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://github.com/MathieuFan22/Mirada-Examen-ML-M2-ISPM.git

fatal: destination path 'Mirada-Examen-ML-M2-ISPM' already exists and is not an empty directory.


In [14]:
!ls

Mirada-Examen-ML-M2-ISPM  sample_data
